In [ ]:
import pandas as pd
msg = pd.read_csv('/content/drive/MyDrive/GAIP PROJECT/Final Dataset/Full_and_Final_dataset.csv')

In [ ]:
msg

,Body,Label
0,Large collection of free and quality porn gall...,Adult
1,Naked Babes Hot Nude Babes pics daily fresh di...,Adult
2,Free porn Japanese Mom Casting Stepmom Teen An...,Adult
3,book WELL THIS IS EXHAUSTING sex columnist new...,Adult
4,The Best Porn Reviews membership Discounts Kno...,Adult
...,...,...
4806,“intern” roles: 5 new jobs - Your job alert fo...,newsLetter
4807,“intern” roles: 30+ new jobs - Your job alert ...,newsLetter
4808,“intern” roles: 3 new jobs - Your job alert fo...,newsLetter
4809,UTS is visiting India! - %%=V(@subjectline)=%...,newsLetter


In [ ]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from nltk.corpus import stopwords

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
corpus = []
for i in range(0, len(msg)):
    # Convert to string in case of non-string values
    review = str(msg['Body'][i])

    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()

    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus

## Tokenization using TF-IDF

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
msg.Label = le.fit_transform(msg.Label)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500, ngram_range = (1,2))
X = tv.fit_transform(corpus).toarray()

In [ ]:
y = msg['Label']
y

0       0
1       0
2       0
3       0
4       0
       ..
4806    3
4807    3
4808    3
4809    3
4810    3
Name: Label, Length: 4811, dtype: int64

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = classifier.predict(X_test)
print('Accuracy of Testing :',accuracy_score(y_pred,y_test))


Accuracy of Testing : 0.9927310488058152


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       159
           1       0.99      0.96      0.98       171
           2       0.99      1.00      0.99       138
           3       1.00      1.00      1.00       495

    accuracy                           0.99       963
   macro avg       0.99      0.99      0.99       963
weighted avg       0.99      0.99      0.99       963



## Testing the model Manually

In [ ]:
def process_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords.words('english')]
    text = ' '.join(text)
    return text

# New text to predict
# Taking a string input from the user
bod = input("Enter the body : ")
new_text = process_text(bod)
new_text_vectorized = tv.transform([new_text]).toarray()

# Predicting the category
y_pred_new = classifier.predict(new_text_vectorized)

Enter the body : this is for you  My Dear ,  I'm glad to inform you about my success in getting those funds transferred under the cooperation of a new partner from bangkok.  Presently i'm in bangkok for investment projects with my own share of the total sum. mean while,i never forget your past efforts and attempts to assist me in transferring those funds despite that it failed us some how.  Now contact my secretary , his name is mr issiac ahokou through his e-mail (  issiac_ahokou@yahoo.fr) ask him to send you the total of $800.000.00 which i kept for your compensation for all the past efforts and attempts to assist me in this matter. I appreciated your efforts at that time very much. So feel free and get intouched with my secretary  issiac and instruct him where to send the amount to you.  Please do let me know immediately you receive it ,so that we can share the joy after all the sufferness at that time. At the moment, I am very busy here because of the investment projects which I an

In [ ]:
def classify_message(code):
    classifications = {
        0: "Adult Spam",
        1: "Fraud",
        2: "Fake Job",
        3: "Newsletter"
    }

    return classifications.get(code, "Unknown Code")

# Testing the function with different codes
test_codes = y_pred_new
for code in test_codes:
    print(f"It belongs to this category: {classify_message(code)}")


It belongs to this category: Fraud


In [ ]:
classifications = {
        0: "Adult Spam",
        1: "Fraud",
        2: "Fake Job",
        3: "Newsletter"
    }

In [ ]:
import joblib

joblib.dump(classifier, 'classification.pkl')

['classification.pkl']

In [ ]:
import joblib

In [ ]:
joblib.dump()

In [ ]:
Y_preds =  classifier.predict(X)
Y_preds

array([0, 0, 0, ..., 3, 3, 3])

In [ ]:
labels = [classifications[x] for x in Y_preds]
labels

In [ ]:
msg["Predicted"] = Y_preds
msg

,Body,Label,Predicted
0,Large collection of free and quality porn gall...,0,0
1,Naked Babes Hot Nude Babes pics daily fresh di...,0,0
2,Free porn Japanese Mom Casting Stepmom Teen An...,0,0
3,book WELL THIS IS EXHAUSTING sex columnist new...,0,0
4,The Best Porn Reviews membership Discounts Kno...,0,0
...,...,...,...
4806,“intern” roles: 5 new jobs - Your job alert fo...,3,3
4807,“intern” roles: 30+ new jobs - Your job alert ...,3,3
4808,“intern” roles: 3 new jobs - Your job alert fo...,3,3
4809,UTS is visiting India! - %%=V(@subjectline)=%...,3,3
